<a href="https://colab.research.google.com/github/ankit-rathi/Tradevesting/blob/main/myProspects_Valuation_Gap_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
import pytz
tz_NY = pytz.timezone('Asia/Kolkata')
datetime_NY = datetime.datetime.now(tz_NY)
print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

Run date time (IST): 2023-11-27 16:14:56


In [2]:
!pip install yfinance

In [3]:
import pandas as pd
import numpy as np

path_mypf = 'https://raw.githubusercontent.com/ankit-rathi/Data-Science-with-Python/master/Tradevesting/'

mypf = pd.read_csv(path_mypf + 'myProspectsScrips.csv')

#mypf = mypf[(mypf['Criteria'] == 'V40') | (mypf['Criteria'] == 'V40N')]
mypf = mypf[mypf['Criteria'].isin(['V40','V40N','VR','AR', 'V200'])]

stock_n100 = mypf['Symbol'].unique()

exclude = ['ACC','AMBUJACEM','CADILAHC','PGHH','SIEMENS', 'GILLETTE', 'NESTLEIND', 'SANOFI', 'AMARAJABAT', 'SCHAEFFLER ']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol']

#stock_ids

stock_ids = stock_n100[:]
#stock_ids = ['HDFCBANK', 'COLPAL']
print('Stocks to be scanned:',len(stock_ids))

Stocks to be scanned: 134


In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

df_latest_analysis = pd.DataFrame()

for stock_name in stock_ids:


    #print(stock_name)

    link_blueprint = "https://www.screener.in/company/"

    stock_list = ['3MINDIA', 'AUBANK', 'COLPAL', 'GLAND', 'LUXIND', 'WHIRLPOOL' , 'IEX', 'ROSSARI', 'VALIANTORG', 'VINATIORGA', 'TATAELXSI',
                  'AMARAJABAT', 'EQUITAS', 'AAVAS', 'CAMS', 'DREAMFOLKS', 'LICI']

    if stock_name in stock_list:
        actual_link = link_blueprint + stock_name.upper()
    else:
        actual_link = link_blueprint + stock_name.upper()  + "/consolidated/"

    def get_list(actual_link):
        page = requests.get(actual_link)
        link_soup = BeautifulSoup(page.content, 'html.parser')

        test_list = link_soup.find(id="profit-loss").text.replace("  ", "").split('\n')

        while("" in test_list) :
            test_list.remove("")
        while("\r" in test_list) :
            test_list.remove("\r")
        while(" \r" in test_list) :
            test_list.remove(" \r")

        return test_list
    #print(actual_link)
    test_list = get_list(actual_link)

    #print(len(test_list))
    n = len(test_list)

    if n < 100:
        actual_link = link_blueprint + stock_name.upper()
        test_list = get_list(actual_link)
        #print(len(test_list))

    #print(test_list)
    if 'TTM' not in test_list:
        end_year = 'Mar 2023'
    else:
        end_year = 'TTM'

        df_yoy = pd.DataFrame()
        #print(test_list)
        s = test_list.index(end_year) +1 - test_list.index('Mar 2018')

        df_yoy['Year'] = test_list[test_list.index('Mar 2018'):test_list.index(end_year)+1]
        df_yoy['Sales'] = test_list[test_list.index('Expenses\xa0+')-s:test_list.index('Expenses\xa0+')]
        df_yoy['Profit'] = test_list[test_list.index('Tax %')-s:test_list.index('Tax %')]

        df_yoy.set_index('Year', inplace=True)
        df_yoy = df_yoy.T
        df_yoy['Stock'] = stock_name.upper() + '.NS'
        #df_yoy
        df_latest_analysis = pd.concat([df_latest_analysis, df_yoy])

df_latest_analysis.index.names = ['Type']
df_latest_analysis.reset_index(inplace=True)
df_latest_analysis.rename(columns={'TTM': 'Nov 2023'}, inplace=True)
#df_latest_analysis.drop('TTM', axis=1, inplace=True)
cols = [ 'Mar 2018', 'Mar 2019', 'Mar 2020', 'Mar 2021', 'Mar 2022', 'Mar 2023', 'Nov 2023']
for col in cols:
    df_latest_analysis[col] = df_latest_analysis[col].str.replace(',', '').astype(float)

In [5]:
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib
yfin.pdr_override()

stock_df_final = pd.DataFrame()

for stock_name in stock_ids:

  stock_symbol = stock_name + '.NS'
  short_window = 20
  mid_window = 50
  long_window = 200
  moving_avg = 'SMA'

  # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
  stock_df = yfin.Ticker(stock_symbol).history(period='6y', interval='1mo')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
  stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
  stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
  stock_df.dropna(axis = 0, inplace = True) # remove any null rows

  stock_df['Stock'] = stock_symbol

  stock_df['Close'] = round(stock_df['Close'],2)
  stock_df['Mon'] = stock_df.index.month_name().str.slice(stop=3)
  stock_df['Year'] = stock_df.index.strftime('%Y')
  stock_df['Year'] = stock_df['Mon'] + ' ' + stock_df['Year']
  cols = [ 'Year', 'Close']
  stock_df = stock_df[cols]
  stock_df = stock_df[(stock_df['Year'].str.contains("Mar")) | (stock_df['Year'].str.contains("Nov 2023"))]
  stock_df = stock_df.set_index('Year')
  stock_df = stock_df.T
  stock_df['Stock'] = stock_symbol
  stock_df.index.names = ['Type']
  stock_df = stock_df.reset_index()
  stock_df_final = pd.concat([stock_df_final, stock_df])

stock_df_final

ERROR:yfinance:Failed to get ticker 'BAJAJFINSV.NS' reason: HTTPSConnectionPool(host='query2.finance.yahoo.com', port=443): Read timed out. (read timeout=10)


Year,Type,Mar 2018,Mar 2019,Mar 2020,Mar 2021,Mar 2022,Mar 2023,Nov 2023,Stock
0,Close,18667.90,23329.87,18105.45,29268.68,18941.44,22882.53,30448.00,3MINDIA.NS
0,Close,203.15,144.82,113.15,249.70,344.70,283.00,503.50,5PAISA.NS
0,Close,272.25,375.83,378.50,651.85,950.19,516.46,530.75,AARTIIND.NS
0,Close,NaN,1155.80,1188.85,2420.85,2569.45,1610.75,1484.95,AAVAS.NS
0,Close,5202.22,6973.80,14775.46,14573.12,17372.22,21929.65,24000.00,ABBOTINDIA.NS
...,...,...,...,...,...,...,...,...,...
0,Close,309.13,472.52,237.62,350.42,737.80,572.70,614.85,VIPIND.NS
0,Close,NaN,NaN,321.81,458.00,477.58,680.83,866.00,WESTLIFE.NS
0,Close,1481.61,1489.94,1794.54,2207.28,1562.80,1310.90,1565.95,WHIRLPOOL.NS
0,Close,206.38,250.09,193.84,409.06,585.57,365.25,396.00,WIPRO.NS


In [6]:
df_gap_analysis = df_latest_analysis.append(stock_df_final)
df_gap_analysis['Growth%'] = (df_gap_analysis['Nov 2023'] - df_gap_analysis['Mar 2018'])*100/df_gap_analysis['Mar 2018']
df_gap_analysis['Growth%'] = round(df_gap_analysis['Growth%'],2)
df_gap_analysis

Year,Type,Mar 2018,Mar 2019,Mar 2020,Mar 2021,Mar 2022,Mar 2023,Nov 2023,Stock,Growth%
0,Sales,2556.00,2809.00,2765.00,2420.00,3078.00,3733.00,3871.00,3MINDIA.NS,51.45
1,Profit,480.00,497.00,320.00,202.00,360.00,561.00,655.00,3MINDIA.NS,36.46
2,Sales,20.00,61.00,108.00,195.00,298.00,338.00,356.00,5PAISA.NS,1680.00
3,Profit,-33.00,-25.00,-10.00,20.00,18.00,58.00,78.00,5PAISA.NS,-336.36
4,Sales,3806.00,4168.00,4186.00,4506.00,6086.00,6619.00,6191.00,AARTIIND.NS,62.66
...,...,...,...,...,...,...,...,...,...,...
0,Close,309.13,472.52,237.62,350.42,737.80,572.70,614.85,VIPIND.NS,98.90
0,Close,NaN,NaN,321.81,458.00,477.58,680.83,866.00,WESTLIFE.NS,NaN
0,Close,1481.61,1489.94,1794.54,2207.28,1562.80,1310.90,1565.95,WHIRLPOOL.NS,5.69
0,Close,206.38,250.09,193.84,409.06,585.57,365.25,396.00,WIPRO.NS,91.88


In [11]:
stock_df_gap = pd.DataFrame()

for stock_name in stock_ids:

  stock_symbol = stock_name + '.NS'
  #print(stock_symbol)
  tmp_df = df_gap_analysis[df_gap_analysis['Stock'] == stock_symbol]
  try:
    gap_perc = round(tmp_df[tmp_df['Type'] == 'Profit']['Growth%'].values[0]  - tmp_df[tmp_df['Type'] == 'Close']['Growth%'].values[0],2)
  except:
    pass
  #print(gap_perc)
  tmp_df = pd.DataFrame()
  tmp_df['Stock'] = [stock_symbol]
  tmp_df['Gap%'] = [gap_perc]
  stock_df_gap = pd.concat([stock_df_gap, tmp_df])

cols = ['Stock', 'Criteria']
tmp_df = mypf[cols]
stock_df_gap = pd.merge(stock_df_gap, tmp_df,  on='Stock')
stock_df_gap.sort_values(['Gap%'], ascending=[False]).head(30)

,Stock,Gap%,Criteria
13,AXISBANK.NS,3630.40,V40
117,TATACOMM.NS,3133.01,AR
116,TANLA.NS,1859.00,AR
71,J&KBANK.NS,1584.59,VR
133,WSTCSTPAPR.NS,380.97,VR
59,ICICIBANK.NS,162.97,V40
22,BAYERCROP.NS,130.59,V40N
65,INFIBEAM.NS,127.88,AR
64,INDUSINDBK.NS,117.72,VR
48,GICRE.NS,105.91,VR


In [12]:
df_gap_analysis[df_gap_analysis['Stock']  == 'EICHERMOT.NS']

Year,Type,Mar 2018,Mar 2019,Mar 2020,Mar 2021,Mar 2022,Mar 2023,Nov 2023,Stock,Growth%
82,Sales,8965.00,9797.00,9154.00,8720.00,10298.00,14442.00,15626.0,EICHERMOT.NS,74.30
83,Profit,3116.00,3297.00,2355.00,1798.00,2203.00,3800.00,4646.0,EICHERMOT.NS,49.10
0,Close,2717.69,1976.14,1268.75,2542.66,2414.26,2916.66,3846.0,EICHERMOT.NS,41.52
